In [2]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

In [3]:
# A method to fix the API
def fixAPI(API):
    if API[0]=='5':
        API = '0'+API
    return API

In [6]:
# Reading the data
start = time.time()
production_data_train = pd.read_csv("production_data_train.csv",dtype={'API':str})
# production_data_test = pd.read_csv("production_data_test.csv",dtype={'API':str})
end = time.time()
print('ReadData',end-start)

ReadData 0.5360305309295654


In [7]:
# Fix api
start = time.time()
production_data_train["API"] = production_data_train["API"].apply(fixAPI)
end = time.time()
print('FixApi',end-start)

FixApi 0.09400510787963867


In [8]:
# Removing duplicates
start = time.time()
production_data_train = production_data_train.sort_values(['_LastUpdate'])
production_data_train.drop_duplicates(subset=['API','Year','Month'],keep='last',inplace=True)
end = time.time()
print('RemovingDuplicates',end-start)

RemovingDuplicates 0.5250298976898193


In [9]:
# Remove pre-peek month
start = time.time()
maxLiquid = production_data_train.groupby(['API'],as_index=False).agg({'Liquid':'max'})
maxDate = pd.merge(maxLiquid[['API','Liquid']],production_data_train[['API','Liquid','Year','Month']],on=['API','Liquid'],how='left')

max1 = maxDate.groupby(['API','Liquid'],as_index=False).agg({'Year':'min'})
maxDate = pd.merge(maxDate[['API','Liquid','Year','Month']],max1[['API','Liquid','Year']],on=['API','Liquid','Year'],how='right')
maxDate = maxDate.groupby(['API','Liquid','Year'],as_index=False).agg({'Month':'min'})

maxDate = maxDate[['API','Year','Month']]
maxDate = maxDate.rename(columns={"Year":"maxYear",'Month':'maxMonth'})
production_data_train = pd.merge(production_data_train,maxDate[['API','maxYear','maxMonth']],on='API')
production_data_train = production_data_train[((production_data_train['Year']-production_data_train['maxYear'])*12+production_data_train['Month']-production_data_train['maxMonth'])>=0]

end = time.time()
print('RemovePrePeekMonth',end-start)

RemovePrePeekMonth 0.5060291290283203


In [10]:
# Adding Index
start = time.time()
production_data_train["Index"] = (production_data_train["Year"]-production_data_train['maxYear'])*12+production_data_train["Month"]-production_data_train["maxMonth"]+1
end = time.time()
print('IndexColumn',end-start)

IndexColumn 0.0740044116973877


In [11]:
production_data_train

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate,maxYear,maxMonth,Index
0,33053063590000,2015,6,10421,12311,1.181365,2717,0.260724,30,2016-04-06 13:47:11.383,2015,4,3
1,33053063590000,2015,7,9777,11254,1.151069,2399,0.245372,31,2016-04-06 13:47:11.383,2015,4,4
2,33053063590000,2015,8,10613,12188,1.148403,2742,0.258362,31,2016-04-06 13:47:11.383,2015,4,5
3,33053063590000,2015,9,13119,17739,1.352161,2903,0.221282,30,2016-04-06 13:47:11.383,2015,4,6
4,33053063590000,2015,10,10318,12979,1.257899,2286,0.221555,31,2016-04-06 13:47:11.383,2015,4,7
5,33053063590000,2015,11,8126,9642,1.186562,1795,0.220896,30,2016-04-06 13:47:11.383,2015,4,8
6,33053063590000,2016,3,6793,8367,1.231709,1803,0.265420,29,2016-05-05 14:00:57.840,2015,4,12
7,33053063590000,2016,4,7325,11139,1.520683,1798,0.245461,30,2016-06-09 14:00:59.117,2015,4,13
8,33053063590000,2016,5,7381,15601,2.113670,1975,0.267579,31,2016-07-07 14:00:28.580,2015,4,14
9,33053063590000,2015,12,7527,7570,1.005713,1679,0.223064,31,2016-08-11 14:03:14.000,2015,4,9


In [14]:
# Getting the training labels (3_yrs_cum_oil production)
temp = production_data_train[production_data_train["Index"]<=36]
y_train = temp.groupby("API").agg({'Liquid':'sum'})
y_train

,Liquid
API,
05001097720000,65412
05001097780100,84043
05001097800000,13471
05001097820000,12992
05001097830000,15195
05001098050100,176206
05001098450000,180189
05005071800000,126323
05005071890100,66150


In [15]:
# reading the features CSVs
Harmony_data = pd.read_csv("Harmony_data.csv",dtype={'API':str})
IHS_data = pd.read_csv("IHS_data.csv",dtype={'API':str})
# fixing the APIs
Harmony_data["API"] = Harmony_data["API"].apply(fixAPI)
IHS_data["API"] = IHS_data["API"].apply(fixAPI)

In [18]:
# making an initial set of training features with the Harmony data only
temp2 = pd.merge(y_train,Harmony_data,on='API',how = 'left')
temp2.fillna(0,inplace = True)
temp2

,API,Liquid,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT,GOR_30,GOR_60,GOR_90
0,05001097720000,65412,4358,760.4881,934.0413,2841.940260,3126.395857,3270.499449
1,05001097780100,84043,4208,611.6503,546.6923,2420.241440,2878.666923,2799.401068
2,05001097800000,13471,3983,0.0000,0.0000,2076.526858,1289.761230,936.715767
3,05001097820000,12992,4221,801.2611,505.2079,3652.173913,4010.889292,3703.643461
4,05001097830000,15195,4027,0.0000,0.0000,2457.326098,2917.465224,3009.740260
5,05001098050100,176206,8190,1408.8110,0.0000,1598.716588,1599.415748,1249.310295
6,05001098450000,180189,4315,0.0000,0.0000,1407.787232,1394.150831,1314.050753
7,05005071800000,126323,4053,3401.1990,2065.5310,1216.537860,888.993541,1275.127121
8,05005071890100,66150,3745,1742.2890,1807.3310,2059.910523,2045.546119,1690.129579
9,05005071920000,186252,3866,1835.5640,1890.1990,3186.014597,2967.191802,2435.975514


In [22]:
x_train = temp2.drop(['API','Liquid'], axis=1)
x_train

,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT,GOR_30,GOR_60,GOR_90
0,4358,760.4881,934.0413,2841.940260,3126.395857,3270.499449
1,4208,611.6503,546.6923,2420.241440,2878.666923,2799.401068
2,3983,0.0000,0.0000,2076.526858,1289.761230,936.715767
3,4221,801.2611,505.2079,3652.173913,4010.889292,3703.643461
4,4027,0.0000,0.0000,2457.326098,2917.465224,3009.740260
5,8190,1408.8110,0.0000,1598.716588,1599.415748,1249.310295
6,4315,0.0000,0.0000,1407.787232,1394.150831,1314.050753
7,4053,3401.1990,2065.5310,1216.537860,888.993541,1275.127121
8,3745,1742.2890,1807.3310,2059.910523,2045.546119,1690.129579
9,3866,1835.5640,1890.1990,3186.014597,2967.191802,2435.975514


In [31]:
x_train = np.array(x_train)
y_train = np.array(y_train) 
y_train.shape
x_train.shape
print(x_train.shape,y_train.shape)

(7844, 6) (7844, 1)


In [48]:
# creating the Machine Learning model
model = Sequential()
model.add(Dense(output_dim=6, activation='tanh', input_dim=x_train.shape[1]))
model.add(Dense(output_dim=6, activation='tanh'))
model.add(Dense(output_dim=y_train.shape[1], activation='linear'))
adam = Adam(lr=0.1)
model.compile(loss='mae', optimizer=adam)

c:\users\gigabyte\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", input_dim=6, units=6)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gigabyte\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=6)`
  after removing the cwd from sys.path.
c:\users\gigabyte\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  """


In [49]:
# fitting the model
model.fit(x_train,y_train,batch_size = 512, epochs= 20000 , verbose = 1,shuffle = True)

Epoch 1/20000
7844/7844 [==============================] - 0s 36us/step - loss: 128229.8636
Epoch 2/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128218.3493
Epoch 3/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128206.7858
Epoch 4/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128195.2854
Epoch 5/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128183.8665
Epoch 6/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128172.4985
Epoch 7/20000
7844/7844 [==============================] - 0s 1us/step - loss: 128161.1763
Epoch 8/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128149.8744
Epoch 9/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128138.5954
Epoch 10/20000
7844/7844 [==============================] - 0s 2us/step - loss: 128127.3248
Epoch 11/20000
7844/7844 [==============================] - 0s 1us/step - loss: 128116.0

7844/7844 [==============================] - 0s 2us/step - loss: 127241.9867
Epoch 90/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127230.8029
Epoch 91/20000
7844/7844 [==============================] - 0s 1us/step - loss: 127219.6142
Epoch 92/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127208.4219
Epoch 93/20000
7844/7844 [==============================] - 0s 1us/step - loss: 127197.2330
Epoch 94/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127186.0459
Epoch 95/20000
7844/7844 [==============================] - 0s 1us/step - loss: 127174.8580
Epoch 96/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127163.6741
Epoch 97/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127152.4893
Epoch 98/20000
7844/7844 [==============================] - 0s 2us/step - loss: 127141.3034
Epoch 99/20000
7844/7844 [==============================] - 0s 1us/step - loss: 127130.1184
Epo

7844/7844 [==============================] - 0s 2us/step - loss: 126258.0222
Epoch 178/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126246.8378
Epoch 179/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126235.6629
Epoch 180/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126224.4827
Epoch 181/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126213.3046
Epoch 182/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126202.1236
Epoch 183/20000
7844/7844 [==============================] - 0s 1us/step - loss: 126190.9472
Epoch 184/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126179.7697
Epoch 185/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126168.5890
Epoch 186/20000
7844/7844 [==============================] - 0s 2us/step - loss: 126157.4094
Epoch 187/20000
7844/7844 [==============================] - 0s 1us/step - loss: 12614

Epoch 265/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125274.4121
Epoch 266/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125263.2468
Epoch 267/20000
7844/7844 [==============================] - 0s 1us/step - loss: 125252.0682
Epoch 268/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125240.8969
Epoch 269/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125229.7241
Epoch 270/20000
7844/7844 [==============================] - 0s 1us/step - loss: 125218.5551
Epoch 271/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125207.3880
Epoch 272/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125196.2254
Epoch 273/20000
7844/7844 [==============================] - 0s 2us/step - loss: 125185.0500
Epoch 274/20000
7844/7844 [==============================] - 0s 1us/step - loss: 125173.8825
Epoch 275/20000
7844/7844 [==============================] - 0s 2us/st

7844/7844 [==============================] - 0s 2us/step - loss: 124292.4224
Epoch 354/20000
7844/7844 [==============================] - 0s 2us/step - loss: 124281.2519
Epoch 355/20000
7844/7844 [==============================] - 0s 2us/step - loss: 124270.0889
Epoch 356/20000
7844/7844 [==============================] - 0s 1us/step - loss: 124258.9293
Epoch 357/20000
7844/7844 [==============================] - 0s 1us/step - loss: 124247.7672
Epoch 358/20000
7844/7844 [==============================] - 0s 2us/step - loss: 124236.6100
Epoch 359/20000
7844/7844 [==============================] - 0s 1us/step - loss: 124225.4475
Epoch 360/20000
7844/7844 [==============================] - 0s 1us/step - loss: 124214.2835
Epoch 361/20000
7844/7844 [==============================] - 0s 2us/step - loss: 124203.1315
Epoch 362/20000
7844/7844 [==============================] - 0s 2us/step - loss: 124191.9742
Epoch 363/20000
7844/7844 [==============================] - 0s 1us/step - loss: 12418

...

7844/7844 [==============================] - 0s 2us/step - loss: 57876.5950
Epoch 19582/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5942
Epoch 19583/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5985
Epoch 19584/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6018
Epoch 19585/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5969
Epoch 19586/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6043
Epoch 19587/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6000
Epoch 19588/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5941
Epoch 19589/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5963
Epoch 19590/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5943
Epoch 19591/20000
7844/7844 [==============================] - 0s 2us/step - l

7844/7844 [==============================] - 0s 2us/step - loss: 57876.6113
Epoch 19669/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6207
Epoch 19670/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5994
Epoch 19671/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5942
Epoch 19672/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5992
Epoch 19673/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5937
Epoch 19674/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5972
Epoch 19675/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5946
Epoch 19676/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5970
Epoch 19677/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5962
Epoch 19678/20000
7844/7844 [==============================] - 0s 2us/step - l

7844/7844 [==============================] - 0s 2us/step - loss: 57876.5936
Epoch 19756/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5983
Epoch 19757/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5959
Epoch 19758/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5968
Epoch 19759/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5914
Epoch 19760/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5991
Epoch 19761/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5930
Epoch 19762/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6011
Epoch 19763/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5913
Epoch 19764/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6010
Epoch 19765/20000
7844/7844 [==============================] - 0s 2us/step - l

7844/7844 [==============================] - 0s 2us/step - loss: 57876.5960
Epoch 19843/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5948
Epoch 19844/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5967
Epoch 19845/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5938
Epoch 19846/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5993
Epoch 19847/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5979
Epoch 19848/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5920
Epoch 19849/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5951
Epoch 19850/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5995
Epoch 19851/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6043
Epoch 19852/20000
7844/7844 [==============================] - 0s 2us/step - l

7844/7844 [==============================] - 0s 2us/step - loss: 57876.5965
Epoch 19930/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5940
Epoch 19931/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5929
Epoch 19932/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5953
Epoch 19933/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5909
Epoch 19934/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.6197
Epoch 19935/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5956
Epoch 19936/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5925
Epoch 19937/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5904
Epoch 19938/20000
7844/7844 [==============================] - 0s 2us/step - loss: 57876.5921
Epoch 19939/20000
7844/7844 [==============================] - 0s 2us/step - l